In [ ]:
# Setup
from sagemaker import get_execution_role
import sagemaker

sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this notebook instance.
role = get_execution_role()

In [ ]:
!pip install chainer --user

In [ ]:
import chainer

from chainer.datasets import get_cifar10

train, test = get_cifar10()

In [ ]:
import os
import shutil

import numpy as np

In [ ]:
# train and test are both tuples.
train_data = [element[0] for element in train]
train_labels = [element[1] for element in train]

In [ ]:
test_data = [element[0] for element in test]
test_labels = [element[1] for element in test]

In [ ]:
# save files locally on disk (any data structure, in this case it's a list)
os.makedirs('/tmp/data/train_cifar')
os.makedirs('/tmp/data/test_cifar')
np.savez('/tmp/data/train_cifar/train.npz', data=train_data, labels=train_labels)
np.savez('/tmp/data/test_cifar/test.npz', data=test_data, labels=test_labels)

In [ ]:
train_input = sagemaker_session.upload_data(
                      path=os.path.join('/tmp', 'data', 'train_cifar'),
                      key_prefix='notebook/chainer_cifar/train')
test_input = sagemaker_session.upload_data(
                      path=os.path.join('/tmp', 'data', 'test_cifar'),
                      key_prefix='notebook/chainer_cifar/test')

In [ ]:
# delete the directory
shutil.rmtree('/tmp/data')

In [ ]:
print('training data at %s' % train_input)
print('test data at %s' % test_input)

In [ ]:
from sagemaker.chainer.estimator import Chainer

In [ ]:
chainer_estimator = Chainer(entry_point='chainer_cifar_vgg_single_machine.py',
                            source_dir="src",
                            role=role,
                            sagemaker_session=sagemaker_session,
                            train_instance_count=1,
                            train_instance_type='ml.p2.xlarge',
                            hyperparameters={'epochs': 10, 'batch-size': 64},
                            framework_version='5.0.0')

In [ ]:
from sagemaker.tuner import ContinuousParameter

hyperparameter_ranges = {'learning-rate': ContinuousParameter(0.05, 0.06)}

In [ ]:
objective_metric_name = 'Validation-accuracy'
metric_definitions = [{'Name': 'Validation-accuracy', 'Regex': '\[J\d\s+\d\.\d+\s+\d\.\d+\s+\d\.\d+\s+(\d\.\d+)'}]

In [ ]:
max_jobs = 5
max_parallel_jobs = 1

In [ ]:
from sagemaker.tuner import HyperparameterTuner

chainer_tuner = HyperparameterTuner(estimator=chainer_estimator,
                                    objective_metric_name=objective_metric_name,
                                    hyperparameter_ranges=hyperparameter_ranges,
                                    metric_definitions=metric_definitions,
                                    max_jobs=max_jobs,
                                    max_parallel_jobs=max_parallel_jobs)

In [ ]:
chainer_tuner.fit({'train': train_input, 'test': test_input})

In [ ]:
chainer_tuner.wait()

In [ ]:
predictor = chainer_tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [13]:
# Setup a an existing predictor (only if you have an existing prediction endpoint)
# from sagemaker import get_execution_role
# import sagemaker
# from sagemaker.chainer import ChainerPredictor

# sagemaker_session = sagemaker.Session()
# predictor = ChainerPredictor('sagemaker-chainer-191122-1513-004-5859cdb9'
# , sagemaker_session=sagemaker_session)

In [25]:
# Let's do some predictions
from skimage import io
import numpy as np

filenames = ['images/airplane1.png',
             'images/automobile1.png',
             'images/bird1.png',
             'images/cat1.png',
             'images/deer1.png',
             'images/dog1.png',
             'images/frog1.png',
             'images/horse1.png',
             'images/ship1.png',
             'images/truck1.png']

In [17]:
image_data = read_images(filenames)

In [18]:
# the prediction via the endpoint, chainer container for serving prediction
# takes an array of images and run inference on it.
response = predictor.predict(image_data) 
for i, prediction in enumerate(response):
    print('image {}: prediction: {}'.format(i, prediction.argmax(axis=0)))

image 0: prediction: 0
image 1: prediction: 8
image 2: prediction: 2
image 3: prediction: 3
image 4: prediction: 4
image 5: prediction: 5
image 6: prediction: 3
image 7: prediction: 7
image 8: prediction: 8
image 9: prediction: 8
